In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import cv2
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import shutil

# ====================
# Convert Images to CSV with Metadata
# ====================
def images_to_csv_with_metadata(image_folder, output_csv):
    data = []
    for idx, filename in enumerate(sorted(os.listdir(image_folder))):
        if filename.endswith(".png"):
            filepath = os.path.join(image_folder, filename)
            image = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
            image = cv2.resize(image, (128, 128))
            image = image / 255.
            image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-6)
            image = np.uint8(image * 255.)
            image_flat = image.flatten()
            row = [idx, filename] + image_flat.tolist()
            data.append(row)

    num_columns = len(data[0]) - 2 if data else 0
    column_names = ["id", "ImageID"] + [indx for indx in range(num_columns)]
    df = pd.DataFrame(data, columns=column_names)
    df.to_csv(output_csv, index=False)

# ====================
# Custom Dataset
# ====================
class SPADDataset(Dataset):
    def __init__(self, image_dir, depth_dir=None, transform=None):
        self.image_dir = image_dir
        self.depth_dir = depth_dir
        self.transform = transform
        self.images = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        image = Image.open(img_path).convert('L')  # SPAD images (binary/grayscale)

        if self.transform:
            image = self.transform(image)

        if self.depth_dir:
            depth_path = os.path.join(self.depth_dir, self.images[idx])
            depth = Image.open(depth_path).convert('L')
            if self.transform:
                depth = self.transform(depth)
            return image, depth

        return image, self.images[idx]

# ====================
# Custom GoogLeNet Block
# ====================
class InceptionBlock(nn.Module):
    def __init__(self, in_channels):
        super(InceptionBlock, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 32, kernel_size=1)
        self.branch3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.Conv2d(32, 48, kernel_size=3, padding=1)
        )
        self.branch5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=1),
            nn.Conv2d(8, 16, kernel_size=5, padding=2)
        )
        self.branch_pool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 16, kernel_size=1)
        )

    def forward(self, x):
        return torch.cat([
            self.branch1x1(x),
            self.branch3x3(x),
            self.branch5x5(x),
            self.branch_pool(x)
        ], dim=1)

# ====================
# GoogLeNet-Style DepthNet
# ====================
class SPADDepthNet(nn.Module):
    def __init__(self):
        super(SPADDepthNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(3, 2, padding=1)

        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(3, 2, padding=1)

        self.inception3a = InceptionBlock(192)
        self.inception3b = InceptionBlock(112)
        self.pool3 = nn.MaxPool2d(3, 2, padding=1)

        self.inception4a = InceptionBlock(112)
        self.inception4b = InceptionBlock(112)
        self.inception4c = InceptionBlock(112)
        self.pool4 = nn.MaxPool2d(3, 2, padding=1)

        self.final_conv = nn.Conv2d(112, 1, kernel_size=1)
        self.weights = nn.Parameter(torch.ones(5))

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = torch.relu(self.conv2(x))
        x = self.pool2(torch.relu(self.conv3(x)))

        s1 = self.inception3a(x)
        s2 = self.inception3b(s1)
        x = self.pool3(s2)

        s3 = self.inception4a(x)
        s4 = self.inception4b(s3)
        s5 = self.inception4c(s4)

        target_size = s1.shape[2:]
        side_outputs = [
            nn.functional.interpolate(s, size=target_size, mode='bilinear', align_corners=False)
            for s in [s1, s2, s3, s4, s5]
        ]

        combined = sum(w * s for w, s in zip(self.weights, side_outputs))
        out = self.final_conv(combined)
        out = nn.functional.interpolate(out, size=(224, 224), mode='bilinear', align_corners=False)
        return out

# ====================
# Training and Evaluation
# ====================
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0
    for img, depth in tqdm(dataloader):
        img, depth = img.to(device), depth.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, depth)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0
    with torch.no_grad():
        for img, depth in tqdm(dataloader):
            img, depth = img.to(device), depth.to(device)
            output = model(img)
            loss = criterion(output, depth)
            running_loss += loss.item()
    return running_loss / len(dataloader)

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))


# ====================
# Prediction + CSV Generation
# ====================
def predict_and_save(model, dataloader, device, save_path="submission.csv", temp_output_folder="temp_predictions"):
    model.eval()
    os.makedirs(temp_output_folder, exist_ok=True)
    filenames = []

    with torch.no_grad():
        for img, fname in tqdm(dataloader):
            img = img.to(device)
            output = model(img).squeeze().cpu().numpy()  # (B, H, W)

            for i in range(output.shape[0]):
                filename = fname[i]
                pred_image = output[i]
                pred_image = (pred_image - pred_image.min()) / (pred_image.max() - pred_image.min() + 1e-6)
                pred_image = (pred_image * 255).astype(np.uint8)
                save_path_img = os.path.join(temp_output_folder, filename)
                cv2.imwrite(save_path_img, pred_image)
                filenames.append(filename)

    # Convert to CSV using the earlier function
    images_to_csv_with_metadata(temp_output_folder, save_path)

    # Optional: delete temporary folder
    shutil.rmtree(temp_output_folder)
    print(f"Saved predictions to {save_path}")

# ====================
# Main Runner
# ====================
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    base_path = "/kaggle/input/ee-5179-modern-computer-vision-course-competition/competition-data"

    train_dataset = SPADDataset(os.path.join(base_path, "training-images"),
                                os.path.join(base_path, "training-depths"),
                                transform)
    val_dataset = SPADDataset(os.path.join(base_path, "validation-images"),
                              os.path.join(base_path, "validation-depths"),
                              transform)
    test_dataset = SPADDataset(os.path.join(base_path, "testing-images"),
                               None,
                               transform)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    model = SPADDepthNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = RMSELoss()

    for epoch in range(1, 4):
        print(f"\nEpoch {epoch}")
        train_loss = train(model, train_loader, optimizer, criterion, device)
        val_loss = evaluate(model, val_loader, criterion, device)
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    predict_and_save(model, test_loader, device, save_path="submission_incep(3).csv")

if __name__ == "__main__":
    main()



Epoch 1


100%|██████████| 53/53 [00:09<00:00,  5.54it/s]


Train Loss: 0.2909, Val Loss: 0.2245

Epoch 2


100%|██████████| 53/53 [00:04<00:00, 12.43it/s]


Train Loss: 0.2225, Val Loss: 0.2148

Epoch 3


100%|██████████| 53/53 [00:04<00:00, 12.48it/s]


Train Loss: 0.2151, Val Loss: 0.2098


100%|██████████| 53/53 [00:06<00:00,  7.58it/s]


Saved predictions to submission_incep(3).csv
